In [1]:
%run base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
i_X_p, i_X_0s = rig_dyn.forward_kinematics(floating_base = True)

In [3]:
H0 , R0, p0 = pluck.spatial_to_homogeneous(i_X_0s[0])
T0 = cs.vertcat(p0, quatT.rotation_matrix_to_quaternion(R0, order='wxyz'))
T0_euler = cs.vertcat(p0, pluck.rotation_matrix_to_euler(R0, order='xyz'))
dIFF_KinJ0 = cs.jacobian(T0_euler, cs.vertcat(base_ss.p_n , arm_ss.q))

H1 , R1, p1 = pluck.spatial_to_homogeneous(i_X_0s[1])
T1 = cs.vertcat(p1, quatT.rotation_matrix_to_quaternion(R1, order='wxyz'))
T1_euler = cs.vertcat(p1, pluck.rotation_matrix_to_euler(R1, order='xyz'))
dIFF_KinJ1 = cs.jacobian(T1_euler, cs.vertcat(base_ss.p_n , arm_ss.q))

H2 , R2, p2 = pluck.spatial_to_homogeneous(i_X_0s[2])
T2 = cs.vertcat(p2, quatT.rotation_matrix_to_quaternion(R2, order='wxyz'))
T2_euler = cs.vertcat(p2, pluck.rotation_matrix_to_euler(R2, order='xyz'))
dIFF_KinJ2 = cs.jacobian(T2_euler, cs.vertcat(base_ss.p_n , arm_ss.q))

H3 , R3, p3 = pluck.spatial_to_homogeneous(i_X_0s[3])
T3 = cs.vertcat(p3, quatT.rotation_matrix_to_quaternion(R3, order='wxyz'))
T3_euler = cs.vertcat(p3, pluck.rotation_matrix_to_euler(R3, order='xyz'))
dIFF_KinJ3 = cs.jacobian(T3_euler, cs.vertcat(base_ss.p_n , arm_ss.q))

H4 , R4, p4 = pluck.spatial_to_homogeneous(i_X_0s[4])
T4 = cs.vertcat(p4, quatT.rotation_matrix_to_quaternion(R4, order='wxyz'))
T4_euler = cs.vertcat(p4, pluck.rotation_matrix_to_euler(R4, order='xyz'))
dIFF_KinJ4 = cs.jacobian(T4_euler, cs.vertcat(base_ss.p_n , arm_ss.q))

fk_eval = cs.Function("fkeval", [arm_ss.q, base_ss.baseT_xyz, base_ss.baseT_rpy, base_ss.p_n], [T0, T1, T2, T3, T4])
fk_eval_euler = cs.Function("fkeval_euler", [arm_ss.q, base_ss.baseT_xyz, base_ss.baseT_rpy, base_ss.p_n], [T0_euler, T1_euler, T2_euler, T3_euler, T4_euler])
dIFF_KinJ = cs.Function("dIFF_KinJ", [arm_ss.q, base_ss.baseT_xyz, base_ss.baseT_rpy, base_ss.p_n], [dIFF_KinJ0, dIFF_KinJ1, dIFF_KinJ2, dIFF_KinJ3, dIFF_KinJ4])
# fk_eval.save("fk_eval.casadi")

In [4]:
# forward kinematics wrt to NED origin quaternion
T_i = fk_eval([cs.pi, cs.pi, cs.pi, cs.pi],[0.140, 0.000, -0.120], [3.142, 1.000, 0.000],[0.2,1,0.5, 0,0,0])
T_i

(DM([0.328219, 1.00001, 0.372436, -0.479426, 9.76461e-05, 0.877583, 0.00017874]),
 DM([0.311257, 1.00002, 0.337776, 0.877583, 0.00017874, 0.479426, -9.76461e-05]),
 DM([0.167379, 1.00008, 0.292929, -0.000178104, 0.877583, -9.88105e-05, -0.479426]),
 DM([0.150417, 1.00009, 0.25827, -0.239868, 0.760057, 0.438708, -0.415105]),
 DM([0.0664799, 1.00013, 0.204375, -0.239868, 0.760057, 0.438708, -0.415105]))

In [5]:
fk_eval_euler = cs.Function("fkeval_euler", [arm_ss.q, base_ss.baseT_xyz, base_ss.baseT_rpy, base_ss.p_n], [T4_euler])
fk_eval.save("fk_eval.casadi")
dIFF_KinJ = cs.Function("dIFF_KinJ", [arm_ss.q, base_ss.baseT_xyz, base_ss.baseT_rpy, base_ss.p_n], [dIFF_KinJ4])
dIFF_KinJ.save("dIFF_KinJ.casadi")

In [6]:
# forward kinematics wrt to NED origin euler
T_i = fk_eval_euler([cs.pi, cs.pi, cs.pi, cs.pi],[0.140, 0.000, -0.120], [3.142, 1.000, 0.000],[0.2,1,0.5, 0,0,0])
T_i

DM([0.0664799, 1.00013, 0.204375, -3.14084, -1, -1.04657])

In [7]:
dKJ = dIFF_KinJ([cs.pi, cs.pi, cs.pi, cs.pi],[0.140, 0.000, -0.120], [3.142, 1.000, 0.000],[0.2,1,0.5, 0,0,0])
dKJ

DM(
[[1, 4.16287e-17, 2.1684e-17, -5.55112e-17, -0.295625, -0.000132355, -2.86748e-08, -0.133402, 0.0885545, -1.04083e-16], 
 [4.16321e-17, 1, -4.1631e-17, 0.295625, 0, -0.13352, 7.02459e-14, -8.14693e-06, -8.49919e-06, 5.55112e-17], 
 [4.25007e-17, -4.1631e-17, 1, 0.000132355, 0.13352, -2.84146e-17, 4.46584e-08, 0.244777, -0.100899, 8.32667e-17], 
 [00, 00, 00, 1, 0.00117417, -1.55741, -3.82368e-25, 0.00117417, -0.00117908, 00], 
 [00, 00, 00, 0, -1, -0.000753923, 3.24971e-22, -1, 1, 00], 
 [00, 00, 00, 0, 0.00139537, -1.85081, 1, 0.000988026, -0.000992159, -1]])

In [7]:
# # c , cpp or matlab code generation for forward dynamics
# fk_eval.generate("fk_eval_.c")
# os.system(f"gcc -fPIC -shared fk_eval_.c -o libFK.so")